# Regression Exercise - Solutions

California Housing Data

This data set contains information about all the block groups in California from the 1990 Census. In this sample a block group on average includes 1425.5 individuals living in a geographically compact area. 

The task is to aproximate the median house value of each block from the values of the rest of the variables. 

 It has been obtained from the LIACC repository. The original page where the data set can be found is: http://www.liaad.up.pt/~ltorgo/Regression/DataSets.html.
 

The Features:
 
* housingMedianAge: continuous. 
* totalRooms: continuous. 
* totalBedrooms: continuous. 
* population: continuous. 
* households: continuous. 
* medianIncome: continuous. 
* medianHouseValue: continuous. 

## The Data

** Import the cal_housing.csv file with pandas. Separate it into a training (70%) and testing set(30%).**

In [1]:
import pandas as pd

In [2]:
housing = pd.read_csv('cal_housing_clean.csv')

In [3]:
housing.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
0,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


In [4]:
housing.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
housingMedianAge,20640.0,28.639486,12.585558,1.0000,18.0000,29.0000,37.00000,52.0000
totalRooms,20640.0,2635.763081,2181.615252,2.0000,1447.7500,2127.0000,3148.00000,39320.0000
totalBedrooms,20640.0,537.898014,421.247906,1.0000,295.0000,435.0000,647.00000,6445.0000
population,20640.0,1425.476744,1132.462122,3.0000,787.0000,1166.0000,1725.00000,35682.0000
households,20640.0,499.539680,382.329753,1.0000,280.0000,409.0000,605.00000,6082.0000
medianIncome,20640.0,3.870671,1.899822,0.4999,2.5634,3.5348,4.74325,15.0001
medianHouseValue,20640.0,206855.816909,115395.615874,14999.0000,119600.0000,179700.0000,264725.00000,500001.0000


In [5]:
x_data = housing.drop(['medianHouseValue'],axis=1)

In [6]:
y_val = housing['medianHouseValue']

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
X_train, X_test, y_train, y_test = train_test_split(x_data,y_val,test_size=0.3,random_state=101)

### Scale the Feature Data

** Use sklearn preprocessing to create a MinMaxScaler for the feature data. Fit this scaler only to the training data. Then use it to transform X_test and X_train. Then use the scaled X_test and X_train along with pd.Dataframe to re-create two dataframes of scaled data.**

In [9]:
from sklearn.preprocessing import MinMaxScaler

In [10]:
scaler = MinMaxScaler()

In [11]:
scaler.fit(X_train)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [12]:
X_train = pd.DataFrame(data=scaler.transform(X_train),columns = X_train.columns,index=X_train.index)

In [13]:
X_test = pd.DataFrame(data=scaler.transform(X_test),columns = X_test.columns,index=X_test.index)

### Create Feature Columns

** Create the necessary tf.feature_column objects for the estimator. They should all be trated as continuous numeric_columns. **

In [14]:
housing.columns

Index(['housingMedianAge', 'totalRooms', 'totalBedrooms', 'population',
       'households', 'medianIncome', 'medianHouseValue'],
      dtype='object')

In [15]:
import tensorflow as tf

C:\Users\ASUS\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [16]:
age = tf.feature_column.numeric_column('housingMedianAge')
rooms = tf.feature_column.numeric_column('totalRooms')
bedrooms = tf.feature_column.numeric_column('totalBedrooms')
pop = tf.feature_column.numeric_column('population')
households = tf.feature_column.numeric_column('households')
income = tf.feature_column.numeric_column('medianIncome')

In [17]:
feat_cols = [ age,rooms,bedrooms,pop,households,income]

** Create the input function for the estimator object. (play around with batch_size and num_epochs)**

In [18]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train ,batch_size=10,num_epochs=1000,shuffle=True)

** Create the estimator model. Use a DNNRegressor. Play around with the hidden units! **

In [19]:
model = tf.estimator.DNNRegressor(hidden_units=[6,6,6],feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\ASUS\\AppData\\Local\\Temp\\tmpbfuupj2a', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001E2FFDD5EB8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


##### ** Train the model for ~1,000 steps. (Later come back to this and train it for more and check for improvement) **

In [20]:
model.train(input_fn=input_func,steps=25000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\ASUS\AppData\Local\Temp\tmpbfuupj2a\model.ckpt.
INFO:tensorflow:loss = 483715740000.0, step = 0
INFO:tensorflow:global_step/sec: 391.235
INFO:tensorflow:loss = 621848300000.0, step = 100 (0.256 sec)
INFO:tensorflow:global_step/sec: 472.011
INFO:tensorflow:loss = 638055350000.0, step = 200 (0.212 sec)
INFO:tensorflow:global_step/sec: 479.894
INFO:tensorflow:loss = 752127050000.0, step = 300 (0.213 sec)
INFO:tensorflow:global_step/sec: 476.043
INFO:tensorflow:loss = 738570530000.0, step = 400 (0.210 sec)
INFO:tensorflow:global_step/sec: 463.396
INFO:tensorflow:loss = 439798660000.0, step = 500 (0.213 sec)
INFO:tensorflow:global_step/sec: 329.827
INFO:tensorflow:loss = 356033270000.0, step = 600 (0.302

INFO:tensorflow:global_step/sec: 433.093
INFO:tensorflow:loss = 94910400000.0, step = 7700 (0.231 sec)
INFO:tensorflow:global_step/sec: 539.252
INFO:tensorflow:loss = 110932935000.0, step = 7800 (0.186 sec)
INFO:tensorflow:global_step/sec: 381.245
INFO:tensorflow:loss = 53643215000.0, step = 7900 (0.262 sec)
INFO:tensorflow:global_step/sec: 533.338
INFO:tensorflow:loss = 85735710000.0, step = 8000 (0.187 sec)
INFO:tensorflow:global_step/sec: 527.724
INFO:tensorflow:loss = 99216740000.0, step = 8100 (0.190 sec)
INFO:tensorflow:global_step/sec: 486.735
INFO:tensorflow:loss = 117425504000.0, step = 8200 (0.204 sec)
INFO:tensorflow:global_step/sec: 335.69
INFO:tensorflow:loss = 76987800000.0, step = 8300 (0.299 sec)
INFO:tensorflow:global_step/sec: 404.302
INFO:tensorflow:loss = 145986910000.0, step = 8400 (0.248 sec)
INFO:tensorflow:global_step/sec: 430.671
INFO:tensorflow:loss = 102523765000.0, step = 8500 (0.231 sec)
INFO:tensorflow:global_step/sec: 412.794
INFO:tensorflow:loss = 764357

INFO:tensorflow:global_step/sec: 430.219
INFO:tensorflow:loss = 48050876000.0, step = 15600 (0.230 sec)
INFO:tensorflow:global_step/sec: 422.757
INFO:tensorflow:loss = 39705793000.0, step = 15700 (0.238 sec)
INFO:tensorflow:global_step/sec: 434.238
INFO:tensorflow:loss = 79734330000.0, step = 15800 (0.234 sec)
INFO:tensorflow:global_step/sec: 416.889
INFO:tensorflow:loss = 159341870000.0, step = 15900 (0.235 sec)
INFO:tensorflow:global_step/sec: 427.937
INFO:tensorflow:loss = 73730290000.0, step = 16000 (0.235 sec)
INFO:tensorflow:global_step/sec: 332.213
INFO:tensorflow:loss = 66485133000.0, step = 16100 (0.302 sec)
INFO:tensorflow:global_step/sec: 397.973
INFO:tensorflow:loss = 46202024000.0, step = 16200 (0.254 sec)
INFO:tensorflow:global_step/sec: 424.545
INFO:tensorflow:loss = 39053435000.0, step = 16300 (0.231 sec)
INFO:tensorflow:global_step/sec: 431.159
INFO:tensorflow:loss = 98709160000.0, step = 16400 (0.232 sec)
INFO:tensorflow:global_step/sec: 430.093
INFO:tensorflow:loss =

INFO:tensorflow:global_step/sec: 397.635
INFO:tensorflow:loss = 85922860000.0, step = 23500 (0.251 sec)
INFO:tensorflow:global_step/sec: 415.895
INFO:tensorflow:loss = 69702480000.0, step = 23600 (0.240 sec)
INFO:tensorflow:global_step/sec: 407.915
INFO:tensorflow:loss = 260535320000.0, step = 23700 (0.240 sec)
INFO:tensorflow:global_step/sec: 430.633
INFO:tensorflow:loss = 63334160000.0, step = 23800 (0.232 sec)
INFO:tensorflow:global_step/sec: 430.785
INFO:tensorflow:loss = 80423540000.0, step = 23900 (0.232 sec)
INFO:tensorflow:global_step/sec: 421.892
INFO:tensorflow:loss = 65547866000.0, step = 24000 (0.237 sec)
INFO:tensorflow:global_step/sec: 435.017
INFO:tensorflow:loss = 57869610000.0, step = 24100 (0.235 sec)
INFO:tensorflow:global_step/sec: 416.317
INFO:tensorflow:loss = 36019896000.0, step = 24200 (0.340 sec)
INFO:tensorflow:global_step/sec: 264.293
INFO:tensorflow:loss = 114399250000.0, step = 24300 (0.274 sec)
INFO:tensorflow:global_step/sec: 442.761
INFO:tensorflow:loss 

** Create a prediction input function and then use the .predict method off your estimator model to create a list or predictions on your test data. **

In [21]:
predict_input_func = tf.estimator.inputs.pandas_input_fn(
      x=X_test,
      batch_size=10,
      num_epochs=1,
      shuffle=False)

In [22]:
pred_gen = model.predict(predict_input_func)

In [23]:
predictions = list(pred_gen)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\ASUS\AppData\Local\Temp\tmpbfuupj2a\model.ckpt-25000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


** Calculate the RMSE. Do this manually or use [sklearn.metrics](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) **

In [24]:
final_preds = []
for pred in predictions:
    final_preds.append(pred['predictions'])

In [25]:
from sklearn.metrics import mean_squared_error

In [26]:
mean_squared_error(y_test,final_preds)**0.5

98466.61728586984

# Great Job!